In [1]:
import tensorflow as tf
print ("TensorFlow version: " + tf.__version__)
import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
import numpy as np
from keras import layers
from keras.models import Model
from keras.models import load_model
from keras.layers.core import Dense, Dropout, Activation, Reshape
from keras import callbacks


TensorFlow version: 2.2.0


In [2]:
xTemp = np.zeros((1060, 6, 340, 444))
with open("C:\\Users\\yangz\\Desktop\\FireProjectBatchDownloadNew\\DetectionDatasetFinal\\TrainingA_1-1060.npy", 'rb') as f1:
        xTemp = np.load(f1)


In [3]:
yTemp = np.zeros((1060))
for i in range (1060):
    if (i%2==0):
        yTemp[i] = 0
    else:
        yTemp[i] = 1


In [4]:
x = np.zeros((1060, 6, 340, 444))
y = np.zeros((1060))

permutations = np.random.permutation(1060)
for i in range (1060):
    x[i] = xTemp[permutations[i]]
    y[i] = yTemp[permutations[i]]


In [5]:
'''
for image in range (1060):
    for channel in range (6):
        for a in range (340):
            for b in range (444):
                temp = int(x[image][channel][a][b]*100000)
                x[image][channel][a][b] = temp
'''

'\nfor image in range (1060):\n    for channel in range (6):\n        for a in range (340):\n            for b in range (444):\n                temp = int(x[image][channel][a][b]*100000)\n                x[image][channel][a][b] = temp\n'

In [6]:
#x = x*100000

In [7]:
#x = x.astype(int)

In [8]:
x = np.reshape(x, (1060, 340, 444, 6))

In [9]:
y = y.astype(int)

In [10]:
img_shape = (340, 444, 6)

In [11]:
validation = np.zeros((212, 6, 340, 444))
with open("C:\\Users\\yangz\\Desktop\\FireProjectBatchDownloadNew\\DetectionDatasetFinal\\Validation_1908-2120.npy", 'rb') as f1:
        validation = np.load(f1)

In [12]:
validation = np.reshape(validation, (212, 340, 444, 6))
yValidation = np.zeros((212))
for i in range (212):
    if (i%2==0):
        yValidation[i] = 0
    else:
        yValidation[i] = 1

In [13]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import roc_auc_score
import sys

space = {
            'units1': hp.choice('units1', [16, 32, 64, 128, 256]),
            'units2': hp.choice('units2', [16, 32, 64, 128, 256]),
            'units3': hp.choice('units3', [16, 32, 64, 128, 256]),
            'units4': hp.choice('units4', [16, 32, 64, 128, 256]),
            'units5': hp.choice('units5', [16, 32, 64, 128, 256]),
    
            'poolSize1': hp.choice('poolSize1', [3, 5]),
            'poolSize2': hp.choice('poolSize2', [3, 5]),
            'poolSize3': hp.choice('poolSize3', [3, 5]),
            'poolSize4': hp.choice('poolSize4', [3, 5]),

            'dropout1': hp.uniform('dropout1', .20,.70),

            'batchSize' : hp.choice('batchSize', [4, 8, 16, 32]),

            'nbEpochs' : hp.choice('nbEpochs', [7, 8, 9]),
    
            'learnRate1' : hp.choice('learnRate1', [1e-3, 1e-4, 1e-5]), 
        }


In [14]:
from tqdm.keras import TqdmCallback

def f_nn(params):
    img = layers.Input(shape=img_shape)

    bn1 = layers.BatchNormalization()(img)
    conv1 = layers.Conv2D(params['units1'], (params['poolSize1'], params['poolSize1']), padding='same', activation='relu', strides=5)(bn1)
    maxPooling1 = layers.MaxPooling2D(padding='same')(conv1)

    bn2 = layers.BatchNormalization()(maxPooling1)
    conv2 = layers.Conv2D(params['units2'], (params['poolSize2'], params['poolSize2']), padding='same', activation='elu', strides=3)(bn2)
    maxPooling2 = layers.MaxPooling2D(padding='same')(conv2)

    bn3 = layers.BatchNormalization()(maxPooling2)
    conv3 = layers.Conv2D(params['units3'], (params['poolSize3'], params['poolSize3']), padding='same', activation='elu', strides=3)(bn3)
    maxPooling3 = layers.MaxPooling2D(padding='same')(conv3)

    bn4 = layers.BatchNormalization()(maxPooling3)
    conv4 = layers.Conv2D(params['units4'], (params['poolSize4'], params['poolSize4']), padding='same', activation='elu', strides=3)(bn4)

    flatten = layers.Flatten()(conv4)

    dense1 = layers.Dense(params['units5'], activation = "relu")(flatten)
    drop = layers.Dropout(params['dropout1'])(dense1)
    res = layers.Dense(1, activation='sigmoid')(drop)

    model = Model(img, res)
    opt = tf.keras.optimizers.Adam(learning_rate=params['learnRate1'])
    model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

    model.fit(x, y, batch_size = int(params['batchSize']), epochs = params['nbEpochs'], verbose=0, callbacks=[TqdmCallback(verbose=1)])
    
    yPred = model.predict(validation)
    yPred[yPred>=0.5] = 1
    yPred[yPred<0.5] = 0
    yPred = yPred.astype(int)
    yPred = yPred.flatten()
    
    correct = 0
    for i in range (212):
        if (yPred[i]==yValidation[i]):
            correct+=1
    acc = correct/212
    
    print('Accuracy:', acc)
    sys.stdout.flush() 
    return {'loss': -acc, 'status': STATUS_OK}

trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=20, trials=trials)
print('best: ', best)


  0%|                                                                           | 0/20 [00:00<?, ?trial/s, best loss=?]

Accuracy:                                                                                                              
0.7641509433962265                                                                                                     
  5%|██▎                                          | 1/20 [06:07<1:56:31, 367.98s/trial, best loss: -0.7641509433962265]

Accuracy:                                                                                                              
0.6320754716981132                                                                                                     
 10%|████▌                                        | 2/20 [11:49<1:48:00, 360.00s/trial, best loss: -0.7641509433962265]

Accuracy:                                                                                                              
0.6981132075471698                                                                                                     
 15%|██████▊                                      | 3/20 [18:16<1:44:16, 368.04s/trial, best loss: -0.7641509433962265]

Accuracy:                                                                                                              
0.5                                                                                                                    
 20%|█████████                                    | 4/20 [24:45<1:39:52, 374.54s/trial, best loss: -0.7641509433962265]

...

Accuracy:                                                                                                              
0.7971698113207547                                                                                                     
 80%|███████████████████████████████████▏        | 16/20 [1:47:58<27:56, 419.09s/trial, best loss: -0.8066037735849056]

Accuracy:                                                                                                              
0.6933962264150944                                                                                                     
 85%|█████████████████████████████████████▍      | 17/20 [1:53:29<19:38, 392.83s/trial, best loss: -0.8066037735849056]

Accuracy:                                                                                                              
0.7547169811320755                                                                                                     
 90%|███████████████████████████████████████▌    | 18/20 [2:01:06<13:44, 412.05s/trial, best loss: -0.8066037735849056]

Accuracy:                                                                                                              
0.6745283018867925                                                                                                     
 95%|█████████████████████████████████████████▊  | 19/20 [2:06:23<06:23, 383.33s/trial, best loss: -0.8066037735849056]

Accuracy:                                                                                                              
0.6320754716981132                                                                                                     
100%|████████████████████████████████████████████| 20/20 [2:13:50<00:00, 401.51s/trial, best loss: -0.8066037735849056]
best:  {'batchSize': 2, 'dropout1': 0.3593679575835619, 'learnRate1': 1, 'nbEpochs': 1, 'poolSize1': 1, 'poolSize2': 1, 'poolSize3': 0, 'poolSize4': 1, 'units1': 0, 'units2': 3, 'units3': 1, 'units4': 0, 'units5': 1}


In [ ]:
#Acc1: 1.00 / 0.7688679245283019